## 40 Objetos com texturas e height maps

As imagens usadas neste notebook foram obtidas no site:

https://3dtextures.me/


In [5]:
import glm
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.PhongShadingShaderProgram_v2 import PhongShadingShaderProgram
from cg.renderers.ModelRenderer_v4 import ModelRenderer
from cg.models.SphereMesh_v3 import SphereMesh
from cg.models.SquareMesh_v3 import SquareMesh
from cg.utils.Texture import Texture

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        self.cameraEye   = glm.vec3(0.0, 0.0, 2.8)
        self.cameraCenter = glm.vec3(0.0, 0.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
        self.viewMatrix = glm.lookAt(self.cameraEye, self.cameraCenter, self.cameraUp)
        
        # cria as malha
        sphere_mesh = []
        for i in range(3):
            sphere_mesh.append(SphereMesh(0.3, 300, 300))
        
        # aplica o height map à malha de triângulos das esferas com uma altura 0.2
        sphere_mesh[0].applyHeightMap('./cg/images/textures/Colored_Pencils_001_SD/Colored_Pencils_001_height.png', 0.2)
        sphere_mesh[1].applyHeightMap('./cg/images/textures/Metal_Sci-fi_003_SD/Metal_Sci-fi_003_height.png', 0.2)
        sphere_mesh[2].applyHeightMap('./cg/images/textures/Pebbles_021_SD/Pebbles_021_4K_Height.png', 0.2)
        
        square_mesh = []
        for i in range(3):
            square_mesh.append(SquareMesh(2.0, 2.0, 300, 300))
        
        # aplica o height map à malha de triângulos dos quadrados com uma altura h
        square_mesh[0].applyHeightMap('./cg/images/textures/Lava_005_SD/Lava_005_DISP.png', 0.1)
        square_mesh[1].applyHeightMap('./cg/images/textures/Brick_Wall_018_SD/Brick_Wall_018_height.png', 0.1)
        square_mesh[2].applyHeightMap('./cg/images/textures/Incrusted_Gems_001_SD/Incrusted_Gems_001_DISP.jpg', 0.2)
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        # cria um objeto de renderização para cada program shader
        self.sphereRenderer = []
        for i in range(3):
            self.sphereRenderer.append(ModelRenderer(sphere_mesh[i].getVertexPositions(),
                                                     vertex_indices=sphere_mesh[i].getVertexIndices(),
                                                     vertex_normal=sphere_mesh[i].getVertexNormals(),
                                                     vertex_tex=sphere_mesh[i].getVertexTextureCoord()))
        
        self.squareRenderer = []
        for i in range(3):
            self.squareRenderer.append(ModelRenderer(square_mesh[i].getVertexPositions(),
                                                     vertex_indices=square_mesh[i].getVertexIndices(),
                                                     vertex_normal=square_mesh[i].getVertexNormals(),
                                                     vertex_tex=square_mesh[i].getVertexTextureCoord()))
        
        # posição da luz
        self.lightPosition = glm.vec4(0.0, 0, 0.5, 1)
        
        # cria o shader program
        self.phongShaderProgram = PhongShadingShaderProgram()
        
        # configura cada shader programa
        self.phongShaderProgram.bind()
        self.phongShaderProgram.setUniformLightPosition(self.viewMatrix * self.lightPosition) #a posição da luz tem que estar no espaço de visão
        self.phongShaderProgram.release()
        
        # recupera o endereço da variável de entrada de cada shader program
        # configura os dados do modelo para serem os dados de entrada do shader program
        position_loc = self.phongShaderProgram.getVertexPositionLoc()
        normal_loc = self.phongShaderProgram.getVertexNormalLoc()
        tex_coord_loc = self.phongShaderProgram.getVertexTextureCoordLoc()
        
        for i in range(3):
            self.sphereRenderer[i].setVertexPositionLoc(position_loc)
            self.sphereRenderer[i].setVertexNormalLoc(normal_loc)
            self.sphereRenderer[i].setVertexTextureCoordLoc(tex_coord_loc)
        
        for i in range(3):
            self.squareRenderer[i].setVertexPositionLoc(position_loc)
            self.squareRenderer[i].setVertexNormalLoc(normal_loc)
            self.squareRenderer[i].setVertexTextureCoordLoc(tex_coord_loc)

        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        # carrega e cria as texturas
        self.textureObject1 = Texture('./cg/images/textures/Colored_Pencils_001_SD/Colored_Pencils_001_basecolor.jpg')
        self.textureObject2 = Texture('./cg/images/textures/Metal_Sci-fi_003_SD/Metal_Sci-fi_003_basecolor.jpg')
        self.textureObject3 = Texture('./cg/images/textures/Pebbles_021_SD/Pebbles_021_4K_basecolor.jpg')
        
        self.textureWall1 = Texture('./cg/images/textures/Lava_005_SD/Lava_005_COLOR.jpg')
        self.textureWall2 = Texture('./cg/images/textures/Brick_Wall_018_SD/Brick_Wall_018_basecolor.jpg')
        self.textureWall3 = Texture('./cg/images/textures/Incrusted_Gems_001_SD/Incrusted_Gems_001_COLOR.jpg')
        
        # inicializa a variável que contém o ângulo de rotação
        self.angle = 0.0
        
    def paintGL(self):
        
        # incrementa a variável que contém o ângulo de rotação
        self.angle += 0.1
        
        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.phongShaderProgram.bind()
        
        # renderiza a cena
        self.renderObject1()
        self.renderObject2()
        self.renderObject3()
        self.renderWall1()
        self.renderWall2()
        self.renderWall3()
        
        # desativa o shader program
        self.phongShaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
        
    def renderObject1(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        transl = glm.translate(glm.mat4(), glm.vec3(0.5, 0.4, 0.0))
        model_matrix = glm.rotate(transl, glm.radians(-self.angle), glm.vec3(0.0, 1.0, 0.0))
        self.phongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.phongShaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.phongShaderProgram.bindTexture2D(self.textureObject1.getTextureID())
        
        # muda a componente especular
        self.phongShaderProgram.setUniformMaterialSpecular(np.array([0.1, 0.1, 0.1]))
        
        # renderiza o objeto
        self.sphereRenderer[0].render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.phongShaderProgram.releaseTexture2D()
    
    def renderObject2(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        transl = glm.translate(glm.mat4(), glm.vec3(-0.5, 0.4, 0.0))
        model_matrix = glm.rotate(transl, glm.radians(-self.angle), glm.vec3(0.0, 1.0, 0.0))
        self.phongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.phongShaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.phongShaderProgram.bindTexture2D(self.textureObject2.getTextureID())
        
        # muda a componente especular
        self.phongShaderProgram.setUniformMaterialSpecular(np.array([0.1, 0.1, 0.1]))
        
        # renderiza o objeto
        self.sphereRenderer[1].render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.phongShaderProgram.releaseTexture2D()
        
    def renderObject3(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        transl = glm.translate(glm.mat4(), glm.vec3(0.0, -0.5, 0.0))
        model_matrix = glm.rotate(transl, glm.radians(-self.angle), glm.vec3(0.0, 1.0, 0.0))
        self.phongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.phongShaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.phongShaderProgram.bindTexture2D(self.textureObject3.getTextureID())
        
        # muda a componente especular
        self.phongShaderProgram.setUniformMaterialSpecular(np.array([0.1, 0.1, 0.1]))
        
        # renderiza o objeto
        self.sphereRenderer[2].render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.phongShaderProgram.releaseTexture2D()
        
    def renderWall1(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, -1.0))
        self.phongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.phongShaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.phongShaderProgram.bindTexture2D(self.textureWall1.getTextureID())
        
        # muda a componente especular
        self.phongShaderProgram.setUniformMaterialSpecular(np.array([0.1, 0.1, 0.1]))
        
        # renderiza o objeto
        self.squareRenderer[0].render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.phongShaderProgram.releaseTexture2D()
        
    def renderWall2(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        transl = glm.translate(glm.mat4(), glm.vec3(-1.0, 0.0, 0.0))
        model_matrix = glm.rotate(transl, glm.radians(90), glm.vec3(0.0, 1.0, 0.0))
        self.phongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.phongShaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.phongShaderProgram.bindTexture2D(self.textureWall2.getTextureID())
        
        # muda a componente especular
        self.phongShaderProgram.setUniformMaterialSpecular(np.array([0.1, 0.1, 0.1]))
        
        # renderiza o objeto
        self.squareRenderer[1].render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.phongShaderProgram.releaseTexture2D()

    def renderWall3(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        transl = glm.translate(glm.mat4(), glm.vec3(1.0, 0.0, 0.0))
        model_matrix = glm.rotate(transl, glm.radians(-90), glm.vec3(0.0, 1.0, 0.0))
        self.phongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.phongShaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.phongShaderProgram.bindTexture2D(self.textureWall3.getTextureID())
        
        # muda a componente especular
        self.phongShaderProgram.setUniformMaterialSpecular(np.array([0.6, 0.6, 0.6]))
        
        # renderiza o objeto
        self.squareRenderer[2].render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.phongShaderProgram.releaseTexture2D()
        
    def resizeGL(self, width, height):
        
        gl.glViewport(0, 0, width, height)
        
        # configura a projeção
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 5.0)

    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        # passo 
        step = 0.1
        
        # verifica se foi pressionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            self.lightPosition.y += step

        # verifica se foi pressionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            self.lightPosition.y -= step
        
        # verifica se foi pressionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            self.lightPosition.x -= step
        
        # verifica se foi pressionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            self.lightPosition.x += step
        
        # verifica se foi pressionada a tecla de seta '-'
        elif event.key() == QtCore.Qt.Key_Minus:
            self.lightPosition.z -= step
        
        # verifica se foi pressionada a tecla de seta '+'
        elif event.key() == QtCore.Qt.Key_Plus:
            self.lightPosition.z += step
        
        self.phongShaderProgram.bind()
        self.phongShaderProgram.setUniformLightPosition(self.viewMatrix * self.lightPosition)
        self.phongShaderProgram.release()
        
def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(800, 800)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

TerrainMesh --> Trying to open ./cg/images/textures/Colored_Pencils_001_SD/Colored_Pencils_001_height.png
opened file: size= (1024, 1024) format= PNG mode= RGB
TerrainMesh --> Trying to open ./cg/images/textures/Metal_Sci-fi_003_SD/Metal_Sci-fi_003_height.png
opened file: size= (1024, 1024) format= PNG mode= RGBA
TerrainMesh --> Trying to open ./cg/images/textures/Pebbles_021_SD/Pebbles_021_4K_Height.png
opened file: size= (1024, 1024) format= PNG mode= RGBA
TerrainMesh --> Trying to open ./cg/images/textures/Lava_005_SD/Lava_005_DISP.png
opened file: size= (1024, 1024) format= PNG mode= RGBA
TerrainMesh --> Trying to open ./cg/images/textures/Brick_Wall_018_SD/Brick_Wall_018_height.png
opened file: size= (1024, 1024) format= PNG mode= RGBA
TerrainMesh --> Trying to open ./cg/images/textures/Incrusted_Gems_001_SD/Incrusted_Gems_001_DISP.jpg
opened file: size= (1024, 1024) format= JPEG mode= RGB
trying to open ./cg/images/textures/Colored_Pencils_001_SD/Colored_Pencils_001_basecolor.jpg

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [33]:
! jupyter nbconvert --to python 41_Objetos_com_texturas_e_height_maps.ipynb
%run -i 41_Objetos_com_texturas_e_height_maps.py

[NbConvertApp] Converting notebook 32_Esferas_iluminacao.ipynb to python
[NbConvertApp] Writing 10068 bytes to 32_Esferas_iluminacao.py
